#### Task 1: Simple vector embedding generation

**Objective:**
Generate vector embeddings from text data.

**Task Description:**

- load embedding model into ollama:
    1. Attach ollama container shell
    2. Run command in container `ollama pull pull bge-m3` or send an api call `requests.post("http://localhost:11434/api/pull", json = {"name": "bge-m3",  "stream": False}` via the requests library to download embedding model.
- embed simple text queries

How to select the right embedding model: [MTEB - Massive Text Embedding Benchmark](https://huggingface.co/blog/mteb)

**Useful links:**

- [Ollama REST API](https://www.postman.com/postman-student-programs/ollama-api/documentation/suc47x8/ollama-rest-api)
- [Langchain Ollama Embeddings](https://python.langchain.com/docs/integrations/text_embedding/ollama/)
- [Langchain Chroma](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/)


In [2]:
import requests

In [3]:
# Download embedding model into ollama container
requests.post("http://localhost:11434/api/pull", json = {"name": "bge-m3",  "stream": False})

<Response [200]>

In [4]:
# List all available models in the ollama container
response = requests.get("http://localhost:11434/api/ps")
response.json()

{'models': []}

In [5]:
from langchain_ollama import OllamaEmbeddings

# ADD HERE YOUR CODE
embedding_model = OllamaEmbeddings(model="bge-m3")

In [6]:
text = "This is a test document."

# ADD HERE YOUR CODE
# Perform vector search
query_vector = embedding_model.embed_query(text)

print(f"Embedding vector length: {len(query_vector)}")
print(query_vector[:10])

Embedding vector length: 1024
[-0.050015017, 0.021115491, -0.05706942, -0.005513766, -0.0062966477, -0.028438922, 0.04517284, 0.042702336, -0.0002470305, 0.013070632]


#### Task 2: Generate embedding vectors with custom dataset

**Objective:**
Load custom dataset, preprocess it and generate vector embeddings.

**Task Description:**

- load pdf document "AI_Book.pdf" via langchain document loader: ``PyPDFLoader``
- use RecursiveCharacterTextSplitter to split documents into chunks
- generate embeddings for single documents

**RecursiveCharacterTextSplitter:**

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough. The default list is `["\n\n", "\n", " ", ""]`. This has the effect of trying to keep all paragraphs (and then sentences, and then words) together as long as possible, as those would generically seem to be the strongest semantically related pieces of text.

**Useful links:**

- [Langchain PyPDFLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.pdf.PyPDFLoader.html)
- [Langchain RecursiveCharacterTextSplitter](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html)


In [7]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.schema import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
import re

pdf_doc = "./AI_Book.pdf"

# Create pdf data loader
# ADD HERE YOUR CODE
pdf_doc = "./AI_Book.pdf"
loader = PyPDFLoader(pdf_doc)

# Load and split documents in chunks
# ADD HERE YOUR CODE
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=150, 
    strip_whitespace=True
)
pages_chunked = loader.load_and_split(text_splitter=text_splitter)

print(pages_chunked[0])

# Function to clean text by removing invalid unicode characters, including surrogate pairs
def clean_text(text):
    # Remove surrogate pairs
    text = re.sub(r'[\ud800-\udfff]', '', text)
    # Optionally remove non-ASCII characters (depends on your use case)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    return text

pages_chunked_cleaned = [
    Document(page_content=clean_text(doc.page_content), metadata=doc.metadata)
    for doc in pages_chunked
]

print(pages_chunked_cleaned[0])


page_content='Aurélien Géron
Hands-on Machine Learning with
Scikit-Learn, Keras, and
TensorFlow
Concepts, Tools, and Techniques to
Build Intelligent Systems
SECOND EDITION
Boston Farnham Sebastopol TokyoBeijing Boston Farnham Sebastopol TokyoBeijing' metadata={'source': './AI_Book.pdf', 'page': 2}
page_content='Aurlien Gron
Hands-on Machine Learning with
Scikit-Learn, Keras, and
TensorFlow
Concepts, Tools, and Techniques to
Build Intelligent Systems
SECOND EDITION
Boston Farnham Sebastopol TokyoBeijing Boston Farnham Sebastopol TokyoBeijing' metadata={'source': './AI_Book.pdf', 'page': 2}


In [8]:
print(pages_chunked_cleaned[1])

page_content='978-1-492-03264-9
[LSI]
Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow
by Aurlien Gron
Copyright  2019 Aurlien Gron. All rights reserved.
Printed in the United States of America.
Published by OReilly Media, Inc., 1005 Gravenstein Highway North, Sebastopol, CA 95472.
OReilly books may be purchased for educational, business, or sales promotional use. Online editions are
also available for most titles (http://oreilly.com). For more information, contact our corporate/institutional
sales department: 800-998-9938 or corporate@oreilly.com.
Editor: Nicole Tache
Interior Designer: David Futato
Cover Designer: Karen Montgomery
Illustrator: Rebecca Demarest
June 2019:  Second Edition
Revision History for the Early Release
2018-11-05: First Release
2019-01-24: Second Release
2019-03-07: Third Release
2019-03-29: Fourth Release
2019-04-22: Fifth Release
See http://oreilly.com/catalog/errata.csp?isbn=9781492032649 for release details.' metadata={'source': './AI_Book

In [9]:
print(f"Number of text chunks: {len(pages_chunked_cleaned)}")

Number of text chunks: 1286


#### Task 3: Store vector embeddings from pdf document to ChromaDB vector database.

**Objective:**

Store vector embeddings into ChromaDB to store knowledge.

**Task Description:**

- create chromadb client
- create chromadb collection
- create langchain chroma db client
- store text document chunks and vector embeddings to vector databases

**Useful links:**

- [Langchain How To](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/#initialization-from-client)

In [1]:
from langchain_chroma import Chroma
import chromadb
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

client = chromadb.HttpClient(
    host="localhost",
    port=8000,
    ssl=False,
    headers=None,
    settings=Settings(allow_reset=True, anonymized_telemetry=False),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

collection_name = "ai_model_book"

# ADD HERE YOUR CODE
# Create a collection
try:
    client.delete_collection(name=collection_name)
    print(f"Alte Collection '{collection_name}' gelöscht.")
except Exception:
    print(f"Keine alte Collection '{collection_name}' gefunden. Fahre fort.")

collection = client.get_or_create_collection(name=collection_name)
print(f"Collection '{collection_name}' ist bereit.")

# ADD HERE YOUR CODE
# Create chromadb
vector_db_from_client = Chroma(client=client,
    collection_name=collection_name,
    embedding_function=embedding_model)

ValueError: {"detail":"Not Found"} (trace ID: 0)

In [2]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(pages_chunked_cleaned))]

# ADD HERE YOUR CODE
vector_db_from_client.add_documents(documents=pages_chunked_cleaned, 
    ids=uuids) # that can take a while...

NameError: name 'pages_chunked_cleaned' is not defined

In [ ]:
client.count_collections()

NameError: name 'client' is not defined

In [ ]:
# client.delete_collection("ai_model_book")

In [ ]:
# Get and check the content of the collection
collection.get(include=["documents", "metadatas", "embeddings"])

NameError: name 'collection' is not defined

#### Task 4: Access ChromaDB and perform vector search

**Objective:**

Use query to perform vector search against ChromaDB vector database

**Task Description:**

- define query
- run vector search
- print k=3 most similar documents


**Useful links:**

- [Langchain Query ChromaDB](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/#query-directly)

In [ ]:
search_query = "Types of Machine Learning Systems"

results = vector_db_from_client.similarity_search(query=search_query, k=3)

for res in results:
    print(res.page_content)
    print(res.metadata)
    print("\n")